In [1]:
from sim.ebm.dy import ModelBaseline
from sim.ebm.intv import Intervention
from sim.fn import load_model_baseline
from sim.inputs import load_inputs
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm.notebook import tqdm
import os

In [2]:
coverage = 0.2

intv1 = Intervention.parse_obj({
    'MassACF': {
        'Coverage': coverage,
        'Sens_L': 0,
        'Spec': 1
    }
})

intv2 = Intervention.parse_obj({
    'MassACF': {
        'Coverage': coverage,
    }
})

intvs = [
    (None, 'Baseline'),
    (intv1, 'Without_LTBI'),
    (intv2, 'With_LTBI')
]

In [3]:
model = load_model_baseline('data')

post = pd.read_csv('pars/baseline/Post.csv').iloc[:, 1:]
post = [dict(row) for _, row in post.iterrows()]
post = [model.Inputs.Cascade.prepare_pars(p) for p in post]

In [4]:
mss_pre = list()
mss_intv = list()

for i, p in enumerate(tqdm(post)):
    ys, ms, _ = model.simulate_to_fit(p, np.linspace(2016, 2022, 7))
    mss_pre.append(ms)
    y0 = ys.sol(2022)

    for intv, lab in intvs:
        _, ms_intv, _ = model.simulate_onward(y0, p, intv)
        mss_intv.append(ms_intv.assign(Scenario=lab, Key=i))

mss_pre = pd.concat(mss_pre)
mss_intv = pd.concat(mss_intv)

  0%|          | 0/300 [00:00<?, ?it/s]

In [5]:
os.makedirs('out/intv', exist_ok=True)

mss_pre.to_csv('out/intv/mss_pre.csv')
mss_intv.to_csv('out/intv/mss_intv.csv')